In [116]:
from math import sqrt
import numpy as np

In [ ]:
def get_data(filename):
    dataset = np.read_csv(filename)
    return dataset

In [117]:
def calculate_distance(point, point0):
    distance = 0
    for i in range(len(point)-1):
        distance += (point[i] - point0[i])**2
    return sqrt(distance)

In [118]:
dataset = [[158, 58, 1],
           [158, 59, 1],
           [158, 63, 1],
           [160, 59, 1],
           [160, 60, 1],
           [163, 60, 2],
           [153, 61, 2],
           [163, 61, 2],
           [165, 61, 3],
           [165, 62, 3],
           [165, 63, 3]]

In [119]:
def get_neighbours(train_set, point0, k):
    distances = []
    
    for point in train_set:
        distance = calculate_distance(point0, point)
        distances.append((point, distance))
    #print(distances)
    distances.sort(key = lambda x: x[1])
    #print(distances)
    neighbours = [point[0] for point in distances[0:k]]
    return neighbours

In [120]:
point0 = [164,60]

In [121]:
def predict(train_set, point0, k):
    neighbours = get_neighbours(train_set, point0, k)
    values = [point[-1] for point in neighbours]
    prediction = max(values, key = values.count)
    return prediction

In [122]:
predict(dataset, point0, 5)

3